In [1]:
import pandas as pd
import datetime as dt 
import openpyxl
import time
from datetime import date, datetime, timedelta
import win32com.client as win32
import xlrd
from openpyxl.workbook import Workbook
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Color, Fill
from openpyxl.cell import Cell
from openpyxl import load_workbook
import xlsxwriter

In [2]:
columns = ['Customer','P.O.number','Customer_PO','Packing_number','Container_number','Invoice_number','Order_number','Document_Number','unit_cost','item','schedule_date','actual inventory level','forecast inventory level','quantity','forecast','Location','p','Type']
dfm = pd.DataFrame(columns=columns)

itlsta = pd.read_csv('itemlist.csv')
itlst1 = itlsta[itlsta['Product Group'].notnull()].sort_values(by=['Product Group','Name'])
products = itlst1.Name.tolist()

Warehouse= ["OLD DO NOT USE A Zollinger's",
            'Ogden', 
            'Memphis (3PL)',
            'Import',
            'Transit',
            'WMS Location',
            'Logan-Inspection',
            'Damaged Location BP',
            'Retail Store',
            'BPUT1','BPUT2',
            'CA 3PL',
            'Drew Logistics LLC'
             ]
#removed invoice number 

InventorySite = ['Import', 'Transit','Damaged Location BP','Retail Store',"OLD DO NOT USE A Zollinger's"]

beginning = []
for itm in products:
    for wh in Warehouse:
        beginning.append(('Beginning_Inventory','','','','','','','',0,itm, dt.datetime.today().strftime("%Y-%m-%d"),0,0,0,0,wh,1,''))
dtfm = pd.DataFrame(beginning, columns = ['Customer','P.O.number','Customer_PO','Packing_number','Container_number','Invoice_number','Order_number','Document_Number','unit_cost','Item','schedule_date','actual inventory level','forecast inventory level','quantity','forecast','Location','p','Type']) 

beginninginventory = pd.read_csv("stockledgerMonthTD.csv",skiprows = 6)
beginninginventory.columns = ['Item', 'Location', 'Description', 'Class', 'Department',
       'Subsidiary', 'Beginning Inv Qty On-hand', 'Beginning Average Cost',
       'Beginning Inv On-hand Value', 'Receipts', 'Other Inv Inputs',
       'Total Input Quantity', 'Value of Inputs', 'Last Receipt Date',
       'Sales', 'Other Inv Outputs', 'Total Output Quantity',
       'Value of Outputs', 'LAST SALES DATE', 'Ending Inv Qty On-hand',
       'Ending Average Cost', 'Ending Inv On-hand Value']
beginninginventory['Ending Inv Qty On-hand'] = beginninginventory['Ending Inv Qty On-hand'].astype(str)
beginninginventory['Ending Average Cost'] = beginninginventory['Ending Average Cost'].astype(str)
for bginv in beginninginventory.index.tolist():
    beginninginventory.at[bginv, 'Ending Average Cost'] = beginninginventory.at[bginv, 'Ending Average Cost'].replace('(','')
    beginninginventory.at[bginv, 'Ending Average Cost'] = beginninginventory.at[bginv, 'Ending Average Cost'].replace(')','')    
    beginninginventory.at[bginv, 'Ending Average Cost'] = beginninginventory.at[bginv, 'Ending Average Cost'].replace('$','')
    beginninginventory.at[bginv, 'Ending Inv Qty On-hand'] = beginninginventory.at[bginv, 'Ending Inv Qty On-hand'].replace(',','')
    beginninginventory.at[bginv, 'Ending Inv Qty On-hand'] = beginninginventory.at[bginv, 'Ending Inv Qty On-hand'].replace('nan','0')
beginninginventory['Ending Inv Qty On-hand'] = beginninginventory['Ending Inv Qty On-hand'].astype(int)
beginninginventory['Ending Average Cost'] = beginninginventory['Ending Average Cost'].astype(float)

beginninginventory1 = beginninginventory[beginninginventory['Item'].isin(products)]
beginninginventory2 =beginninginventory1[["Item",'Location',"Ending Inv Qty On-hand",'Ending Average Cost']]
#beginninginventory2['schedule_date'] = (date.today() - timedelta(days=1)).strftime("%m/%d/%Y")
beginninginventory2['Customer'] = 'Beginning Inventory'
beginninginventory2.columns = ["Item",'Location','quantity','unit_cost','Customer']
bi1 = beginninginventory2.merge(dtfm,on=['Item',"Location"],how='right')
bi2 = bi1.fillna('')
bi = bi2[['Customer_x','P.O.number','Packing_number','Customer_PO','Container_number','Invoice_number','Order_number','Document_Number','unit_cost_x','Item','actual inventory level','forecast inventory level','quantity_x','forecast','Location','p']]
bi.columns = ['Customer','P.O.number','Packing_number','Customer_PO','Container_number','Invoice_number','Order_number','Document_Number','unit_cost','Item','actual inventory level','forecast inventory level','quantity','forecast','Location','p']
bi['Customer'] = 'Beginning Inventory'
bi['schedule_date'] = date.today().strftime("%Y-%m-%d")
bi=bi[['Customer','P.O.number','Packing_number','Customer_PO','Container_number','Invoice_number','Order_number','Document_Number','unit_cost','Item','schedule_date','actual inventory level','forecast inventory level','quantity','forecast','Location','p']]
bi['Type'] = ''  

false = pd.read_csv("OpenSalesOrdersWithShipDateResultsmainline=false.csv")

for issd in list(false[false['Ship Date'].notnull()].index.tolist()):
    false.at[issd, 'Date']  = false.at[issd, 'Ship Date']
for isd in list(false[false['Item Schedule Date 1'].notnull()].index.tolist()):
    false.at[isd, 'Ship Date']  = false.at[isd, 'Item Schedule Date 1'] 
    false.at[isd, 'Date']  = false.at[isd, 'Item Schedule Date 1']
true = false[false['Item'] == "-Not Taxable-"]
true.drop_duplicates(subset =["Document Number",'Ship Date'], keep = False, inplace = True)
true1 = true[["Document Number","Ship Date"]]
false['unit_cost'] = false['Amount']/false['Quantity']
false1 = false[false["Item"].isin(products)]
false2 = false1[["Document Number","Ship Date","Name","Item","PO/Check Number","Location","Quantity","unit_cost","Fulfilling/Receiving Transaction","Quantity Committed","Quantity Fulfilled"]]
false2a = false1[["Date","Document Number","Ship Date","Name","Item","PO/Check Number","Location","Quantity","unit_cost","Fulfilling/Receiving Transaction","Quantity Committed","Quantity Fulfilled"]]
false3e = false2a[false2a["Ship Date"].isnull()]
false3e = false3e[["Date","Document Number","Name","Item","PO/Check Number","Location","Quantity","unit_cost","Fulfilling/Receiving Transaction","Quantity Committed","Quantity Fulfilled"]]
false3ne = false2[false2["Ship Date"].notnull()]
false3e = pd.merge(false3e, true1, how='left', left_on='Document Number', right_on='Document Number').reset_index(drop = True)
for nosd in list(false3e[false3e['Ship Date'].isnull()].index.tolist()):
    false3e.at[nosd, 'Ship Date'] = false3e.at[nosd, 'Date']

falsebeta = false1[["Document Number","Name","Status","Item","PO/Check Number","Quantity Committed","Quantity Picked","Quantity Packed","Quantity Fulfilled",]]


false1sn = false1[["Document Number","Item"]].sort_values(by=['Document Number'])
false1snbeta = false1sn.groupby(['Document Number'],as_index=False).count()
false1snbeta.columns = ["Document Number","SO Lines"]


opensalesorderswithdate = false3e.append(false3ne)
opensalesorderswithdate['quantity'] = (opensalesorderswithdate['Quantity'] - opensalesorderswithdate['Quantity Fulfilled']) * (-1)
opensalesorders3 = opensalesorderswithdate[["Document Number","Name","Item","PO/Check Number","Location","quantity","unit_cost","Ship Date","Fulfilling/Receiving Transaction","Quantity","Quantity Fulfilled"]]
opensalesorders3.columns = ["Document_Number","Customer","Item","P.O.number","Location","quantity","unit_cost",'schedule_date','Fulfilling/Receiving Transaction',"Quantity","Quantity Fulfilled"]
opensalesorders3['schedule_date'] = pd.to_datetime(opensalesorders3['schedule_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
opensalesorders3 = opensalesorders3.round({'unit_cost': 2})

opensalesordersns0 = opensalesorders3[(opensalesorders3['Fulfilling/Receiving Transaction']==' ') & (opensalesorders3['schedule_date'] < dt.datetime.today().strftime("%Y-%m-%d"))] 
opensalesordersns1 = opensalesorders3[(opensalesorders3['Fulfilling/Receiving Transaction']!=' ') & (opensalesorders3['Quantity'] > 0)]
opensalesordersns1['quantity'] = (opensalesordersns1["Quantity"] - opensalesordersns1["Quantity Fulfilled"]) * (-1)
opensalesordersns = opensalesordersns1.append(opensalesordersns0,ignore_index=True)
opensalesordersns = opensalesordersns[["Document_Number","Customer","Item","P.O.number","Location","quantity","unit_cost",'schedule_date','Fulfilling/Receiving Transaction']]
opensalesordersns['schedule_date'] = pd.to_datetime(opensalesordersns['schedule_date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
opensalesordersns['Type']= 'unFulfilled S.O.'
opensalesordersns['r'] = 2
opensalesordersns['p'] = 3

opensalesorders = opensalesorders3[(opensalesorders3['Fulfilling/Receiving Transaction']==' ') & (opensalesorders3['schedule_date'] >= dt.datetime.today().strftime("%Y-%m-%d"))]
opensalesorders['Type']= 'S.O.'
opensalesorders = opensalesorders[["Document_Number","Customer","Item","P.O.number","Location","quantity","unit_cost",'schedule_date','Type']]
opensalesorders = opensalesorders.round({'unit_cost': 2})
# some schedule_date is NAT

bioso = bi.append(opensalesorders,sort=True)
bioso.fillna('')
bioso.loc[bioso.quantity == '', 'quantity'] = 0
bioso['schedule_date'] = pd.to_datetime(bioso['schedule_date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
bioso = bioso[bioso['schedule_date'] >= dt.datetime.today().strftime("%Y-%m-%d")]
bioso.sort_values(by = 'schedule_date')

import1 = pd.read_csv('PurchaseOrdersNotShippedDropShip.csv')
import2 = import1[['Name','Purchase Order','Item','Quantity','Customer PO','unit_cost','Blanket PO']]
import2['Item'] = import2['Item'].astype(str)
#################12/9/2020
opensalesorderswithdate1 = opensalesorderswithdate[['PO/Check Number','Item','Ship Date']]
opensalesorderswithdate1.columns = ['Customer PO','Item','schedule_date']

opensalesorderswithdate2 = opensalesorderswithdate1.dropna().drop_duplicates()##############################################
#################12/9/2020
import3 = import2.merge(opensalesorderswithdate2,how='left', on=['Customer PO','Item'])
imnport4 = import3[['Name','Purchase Order','Item','Quantity','Customer PO','schedule_date','unit_cost','Blanket PO']]
imnport4.columns = ['Customer','Customer_PO','Item','quantity','P.O.number','schedule_date','unit_cost','Blanket PO']
#some P.O. has NaT on schedule_date
imnport4['schedule_date'] = pd.to_datetime(imnport4['schedule_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
imnport4["Type"] = 'P.O.'
for blkpo1 in list(imnport4[imnport4['Blanket PO']=='Yes'].index.tolist()):
    imnport4.at[blkpo1, 'Type'] = 'P.O.Blanket'
imnport4 = imnport4[['Customer','Customer_PO','Item','quantity','P.O.number','schedule_date','unit_cost','Type']]

OPONS = pd.read_csv('OpenPurchaseOrdersNotshipped_Xudong.csv')
OPONS['unit_cost'] = OPONS['Amount']/OPONS['Quantity']
OPONS['quantity'] = OPONS['Quantity'] - OPONS['Quantity Fulfilled/Received']
OPONS['Item'] = OPONS['Item'].astype(str)
OPONSConf = OPONS[OPONS['Confirmed'] == 'Yes']
OPONSConf1 = OPONSConf[['Name','Purchase Order','unit_cost','Item','quantity','Destination','Expected Receipt Date','Blanket PO']]
OPONSConf1.columns = ['Customer','Customer_PO','unit_cost','Item','quantity','Location','schedule_date','Blanket PO']
OPONSConf1['Type'] = 'Confirmed P.O.'
for blkpo2 in list(OPONSConf1[OPONSConf1['Blanket PO']=='Yes'].index.tolist()):
    OPONSConf1.at[blkpo2, 'Type'] = 'Confirmed P.O.Blanket'
OPONSConf1 = OPONSConf1[['Customer','Customer_PO','unit_cost','Item','quantity','Location','schedule_date','Type']]
OPONSConf1['schedule_date'] = pd.to_datetime(OPONSConf1['schedule_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

OPONSnonConf = OPONS[OPONS['Confirmed'] == 'No']
OPONSnonConf['unit_cost'] = OPONSnonConf['Amount']/OPONSnonConf['Quantity']
OPONSnonConf1 = OPONSnonConf[['Date','Name','Purchase Order','unit_cost','Item','quantity','Destination','Expected Receipt Date','Blanket PO']]
OPONSnonConf1.columns = ['Date','Customer','Customer_PO','unit_cost','Item','quantity','Location','schedule_date','Blanket PO']
OPONSnonConf1['Date']= pd.to_datetime(OPONSnonConf1['Date'])
OPONSnonConf1['schedule_date'] = pd.to_datetime(OPONSnonConf1['Date'] + timedelta(days=366), format='%m/%d/%Y').dt.strftime('%Y-%m-%d')#.strftime("%Y-%m-%d")
OPONSnonConf1['Type'] = 'Non-Confirmed P.O.'
for blkpo3 in list(OPONSnonConf1[OPONSnonConf1['Blanket PO']=='Yes'].index.tolist()):
    OPONSnonConf1.at[blkpo3, 'Type'] = 'Non-Confirmed P.O.Blanket'
OPONSnonConf1 = OPONSnonConf1[['Customer','Customer_PO','unit_cost','Item','quantity','Location','schedule_date','Type']]


notshiped = OPONSnonConf1.append(OPONSConf1,sort=True)
notshiped.reset_index(drop=True)
notshiped.sort_values(by = 'schedule_date')
notshippedandbioso = notshiped.append(bioso,sort=True,ignore_index=True)
notshippedandbioso.reset_index(drop=True)
dfnb = imnport4.append(notshippedandbioso,sort=True,ignore_index=True)
dfnb.reset_index(drop=True)

backorder1 = pd.read_csv("backorder.csv",skiprows = 6)
backorder1.columns = ['Document Number', 'Item', 'Customer', 'Ordered', 'Fulfilled',
       'Back Ordered', 'On Hand', 'On Order', 'Date', 'Date Shipped',
       'Reference Number', 'Unit Price', 'Location: Name (Grouped)']
for bkdrs1 in list(backorder1[backorder1['Date Shipped'].notnull()].index.tolist()):
    backorder1.at[bkdrs1, 'Date']  = backorder1.at[bkdrs1, 'Date Shipped'] 
    
backorder1['Unit Price'] = backorder1['Unit Price'].astype(str)
backorder1['On Hand'] = backorder1['On Hand'].astype(str)
backorder1['On Order'] = backorder1['On Order'].astype(str)
for bldr in backorder1.index.tolist():
    backorder1.at[bldr, 'Unit Price'] = backorder1.at[bldr, 'Unit Price'].replace('$','')
    backorder1.at[bldr, 'Unit Price'] = backorder1.at[bldr, 'Unit Price'].replace(',','')
    backorder1.at[bldr, 'Back Ordered'] = backorder1.at[bldr, 'Back Ordered'].replace(',','')
    backorder1.at[bldr, 'Back Ordered'] = backorder1.at[bldr, 'Back Ordered'].replace('nan','0')
    backorder1.at[bldr, 'On Hand'] = backorder1.at[bldr, 'On Hand'].replace(',','')
    backorder1.at[bldr, 'On Hand'] = backorder1.at[bldr, 'On Hand'].replace('nan','0')
    backorder1.at[bldr, 'On Order'] = backorder1.at[bldr, 'On Order'].replace(',','')
    backorder1.at[bldr, 'On Order'] = backorder1.at[bldr, 'On Order'].replace('nan','0')
backorder1['On Hand'] = backorder1['On Hand'].astype(int)
backorder1['On Order'] = backorder1['On Order'].astype(int)
backorder1['Back Ordered'] = backorder1['Back Ordered'].astype(int)
backorder1['Unit Price'] = backorder1['Unit Price'].astype(float)
backorder1['p'] = 3
backorder1['r'] = 2



falsebor = false[false['Ship Date'].notnull()]
falsebor1 = falsebor[['Ship Date','Document Number','Item']]
backorder1a = pd.merge(backorder1,falsebor1, how='left', left_on=['Document Number','Item'], right_on=['Document Number','Item'])

for bkdrs in list(backorder1a[backorder1a['Ship Date'].notnull()].index.tolist()):
    backorder1a.at[bkdrs, 'Date Shipped']  = backorder1a.at[bkdrs, 'Ship Date'] 
    backorder1a.at[bkdrs, 'Date']  = backorder1a.at[bkdrs, 'Ship Date']

backorder = backorder1a[['Customer','Document Number','Item','Back Ordered','Reference Number','Unit Price','Date','Location: Name (Grouped)','p','r']]
backorder.columns = ['Customer','Document_Number','Item','quantity','P.O.number','unit_cost','Date','Location','p','r']

############
opensalesorders4 = opensalesorders3[['Document_Number','Item','schedule_date']]
opensalesorders4.columns = ['Document_Number','Item','Ship Date']
opensalesorders4['Ship Date'] = pd.to_datetime(opensalesorders4['Ship Date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y')

backorders = pd.merge(backorder, opensalesorders4, how='left', left_on=['Document_Number','Item'], right_on=['Document_Number','Item'])
###############

for sd in list(backorders[backorders['Ship Date'].isnull()].index.tolist()):
    backorders.at[sd, 'Ship Date'] = backorders.at[sd, 'Date']

backordersss = backorders[['Customer','Document_Number','Item','quantity','P.O.number','unit_cost','Ship Date','Location','p','r']]
backordersss['Item'] = backordersss['Item'].astype(str)
backorderss = backordersss[backordersss['Item'].isin(products)]

backorderss.columns =['Customer','Document_Number','Item','quantity','P.O.number','unit_cost','schedule_date','Location','p','r']
backorderss['schedule_date'] = pd.to_datetime(backorderss['schedule_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
backorderss['quantity'] = -1 * backorderss['quantity']
backorderss['Type'] = 'backorder S.O.'

nsandbk = opensalesordersns.append(backorderss,sort=True,ignore_index=True)
nsandbkdr = nsandbk.sort_values('Type',ascending = False).drop_duplicates(['Document_Number','Item'], keep='last').sort_index()

nsandbkdr1 = nsandbkdr
nsandbkdr1['Customer_PO'] = ''
nsandbkdr2 = nsandbkdr1[nsandbkdr1['Location'].str.lower() == 'import']

import2['compressed']=import2.apply(lambda x:'%s%s' % (x['Item'],x['Customer PO']),axis=1)
nsandbkdr2['compressed']=nsandbkdr2.apply(lambda x:'%s%s' % (x['Item'],x['P.O.number']),axis=1)
import2['Success'] = import2['compressed'].isin(nsandbkdr2['compressed']).astype(int)
import3 = import2[import2['Success']!= 0]
nsandbkdr3 = nsandbkdr2[['compressed','p','r','schedule_date']]
import4 = import3.merge(nsandbkdr3,on = 'compressed')
import5 = import4[['Name','Purchase Order','Item','Quantity','Customer PO','p','r','schedule_date']]
import5.columns = ['Customer','Customer_PO','Item','quantity','P.O.number','p','r','schedule_date']

import5['Type'] = 'P.O.'
import5['Location'] = 'Import'
nsandbkdr2 = nsandbkdr1.append(import5)
nsandbkdr2.reset_index(drop = True)
for tp in list(nsandbkdr2[nsandbkdr2['Type'].isnull()].index.tolist()):
        nsandbkdr2.at[tp, 'Type'] = 'backorderS.O.'


frecst = pd.read_excel('Forecast.xlsx')
#frcst = frecst[(frecst['Location'] != 'Import') & (frecst['Location'] != 'import')]
for frst in list(frecst[frecst['Location'].str.lower() == 'import'].index.tolist()):
        frecst.at[frst, 'Location'] = 'Import'
frcst = frecst
frcst.columns = ['Customer','Item','unit_cost','schedule_date','forecast','Location'] 
frcst['Type'] = 'Forecast'
frcst['schedule_date'] = pd.to_datetime(frcst['schedule_date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
frcst['p'] = 3
frcst['r'] = 2
frcst['quantity'] = 0
frcst['forecast'] = frcst['forecast'].round(0).astype(int)
frcst['forecast'] = frcst['forecast'] * - 1
frcst = frcst.round({'forecast': 0})
frcst['Item'] = frcst['Item'].astype(str)
frcst = frcst[frcst['schedule_date'] >= dt.datetime.today().strftime("%Y-%m-%d")]


shipment = pd.read_csv('ShipmentinTransit.csv')
shipment = shipment[shipment['Quantity'].notnull()]
shipment['schedule_date'] = (pd.to_datetime(shipment['Port ETA (IR)'], format='%m/%d/%Y') + timedelta(days=7))
shipment['schedule_date'] = pd.to_datetime(shipment['schedule_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
shipment['unit_cost'] = shipment['Amount']/shipment['Quantity']
shipment1 = shipment[['Purchase Order','Item Receipt #','unit_cost','Item','Quantity','Container # (IR)','Destination (PO)','schedule_date']]
shipment1['Item'] = shipment1['Item'].astype(str)
shipment2 = shipment1
shipment2.columns =  ['Customer_PO','Document_Number','unit_cost','Item','forecast','Container_number','Location','schedule_date']
shipment2['quantity'] = shipment2['forecast']
shipment2['quantity'] = shipment2['quantity'].astype(int)
shipment2['Type'] = 'Est.Transit Receive'
shipment2['p'] = 3
shipment2['r'] = 2
shipment3 = shipment2[['Item','quantity']]
shipment4 = shipment3.groupby(['Item'],as_index = False).agg({'quantity': ['sum']})
shipment4.columns = shipment4.columns.droplevel(1)

pastPO = OPONSConf1[OPONSConf1['schedule_date'] >= (dt.datetime.today()- timedelta(days=92)).strftime("%Y-%m-%d")]
pastPO = pastPO[pastPO['schedule_date'] < dt.datetime.today().strftime("%Y-%m-%d")]
pastPO = pastPO[pastPO['quantity']>0]
pastPO['Type'] = 'Past 90 Days P.O.'
pastPO['Item'] = pastPO['Item'].astype(str)
pastPO['p'] = 3
pastPO['r'] = 2

dfnb['schedule_date'] = pd.to_datetime(dfnb['schedule_date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

dfini = dfnb
dfini = dfini[dfini['schedule_date'] >= dt.datetime.today().strftime("%Y-%m-%d")]

df11 = dfini.append(shipment2,sort=True,ignore_index=True)
df11.reset_index(drop=True, inplace=True)

df11['p'] = 3
df11['r'] = 2

for beginv in list(df11.Customer[df11['Customer'] == 'Beginning Inventory'].index.tolist()):
        df11.at[beginv, 'p'] = 1

df = df11.append(nsandbkdr2,sort=True,ignore_index=True)

df = df.fillna('')
df.reset_index(drop=True, inplace=True)
df['Customer'] = df['Customer'].astype(str)
#df['schedule_date'] = df['schedule_date'].astype('datetime64[ns]')
#df['vendor_ship_date'] = df['vendor_ship_date'].astype('datetime64[ns]')
df['P.O.number'] = df['P.O.number'].astype(str)
df['Container_number'] = df['Container_number'].astype(str)
df['Customer_PO'] = df['Customer_PO'].astype(str)
df['Packing_number'] = df['Packing_number'].astype(str)
df['Invoice_number'] = df['Invoice_number'].astype(str)
df['Order_number'] = df['Order_number'].astype(str)
df['Document_Number'] = df['Document_Number'].astype(str)
df['Item'] = df['Item'].astype(str)
df['Location'] = df['Location'].astype(str)
df['quantity'] = df['quantity'].replace(',','')
df['quantity'] = df['quantity'].astype(int)
df = df[['Customer','P.O.number','Customer_PO','Container_number',  'Document_Number', 
       'Invoice_number', 'Item', 'Location', 'Order_number', 'Packing_number', 'actual inventory level', 'forecast','Type',
       'forecast inventory level', 'p', 'quantity', 'schedule_date','unit_cost','r']]
#df.groupby(['Location','Item'],as_index=False).agg({'quantity': [sum],'unit_cost':[max]})
df['actual inventory level'] = 0
df['forecast inventory level'] = 0
df = df.round({'unit_cost': 2})

dfim = df[(df['Location'].str.lower() == 'import') & (df['P.O.number']!='')]
for itmlst in list(df[df['P.O.number'].isin(dfim['P.O.number'].unique().tolist())].index.tolist()):
        df.at[itmlst, 'Location'] = 'Import'
dfa = df[df['Type'].isin(['backorderS.O.','S.O.'])]
dfb = df[(df['Type']!='backorderS.O.')&(df['Type']!='S.O.')]
dfa= dfa.sort_values(by=['P.O.number','Item','Type'])
dfa1 = dfa.drop_duplicates(subset = ['P.O.number','Document_Number','Item'],keep = 'first')
dff = dfb.append(dfa1)

dffi = dff[dff['Location'] == 'Import']
dffni = dff[dff['Location'] != 'Import']
######

dfff = dffni.append(dffi)

invcolumns = ['Item','act Inv <0 date','frcst Inv <0 date','on order']
invdate = pd.DataFrame(columns=invcolumns)

dfff.reset_index(drop=True)


falsebeta = false1[["Document Number","Name","Status","Item","PO/Check Number",'Quantity',"Quantity Committed","Quantity Packed","Quantity Fulfilled",]]
falsebetaa = pd.merge(falsebeta, backorder1[['Document Number','Item','Back Ordered']], how='left', left_on=['Document Number','Item'], right_on=['Document Number','Item'])
falsebetaa = falsebetaa.fillna(0)
falsebetaa["Quantity Packed"] = falsebetaa["Quantity Packed"] - falsebetaa["Quantity Fulfilled"]
falsebetaa["Quantity Picked"] = falsebetaa["Quantity"] - falsebetaa["Quantity Fulfilled"] - falsebetaa["Quantity Packed"] - falsebetaa['Back Ordered']
falsebetaa = falsebetaa[["Document Number","Name","Status","Item","PO/Check Number","Quantity Committed","Quantity Picked","Quantity Packed","Quantity Fulfilled",'Back Ordered']]

dfffbeta = pd.merge(dfff, falsebetaa, how='left', left_on=['P.O.number','Document_Number','Item'], right_on=['PO/Check Number','Document Number','Item'])

dfffbeta1 = pd.merge(dfffbeta, false1snbeta, how='left', left_on=['Document_Number'], right_on=['Document Number'])

dfffbeta1 = dfffbeta1[(dfffbeta1['quantity'] != 0)|(dfffbeta1['Type'] == 'Forecast')]

#df = df.sort_values(by = ['Type','Customer','schedule_date','P.O.number','Item','Document_Number'])
#df = df.drop_duplicates(subset = ['Customer','P.O.number','Item'],keep= 'first')
file_name = "NAI_Inventory_Wizard_Plus " + str(dt.datetime.today().strftime("%Y-%m-%d %A %H;%M;%S"))+".xlsx"
#file_name_no_price = "NAI_Inventory_Wizard " + str(dt.datetime.today().strftime("%Y-%m-%d %A %H;%M;%S"))+".xlsx"


#####
dfffbeta2 = dfffbeta1.drop_duplicates(subset=['P.O.number','Customer_PO','SO Lines','Document_Number','Item','schedule_date','Location','quantity','Status'])
#####12/4/2020
dfffbeta2 = dfffbeta2.append(frcst,sort=True,ignore_index=True)
dfffbeta2 = dfffbeta2.append(pastPO,ignore_index=True)

dfffbeta3 = dfffbeta2[(dfffbeta2['Item'].str.len() == 4)]
dfffbeta4 = pd.merge(dfffbeta3[['Item']], itlst1[['Product Group','Name']],  how='left', left_on=['Item'], right_on = ['Name'])
dfffbeta4 = dfffbeta4.sort_values(by=['Product Group','Name'])
dfffbeta4 = dfffbeta4.drop_duplicates(subset = ['Name'])
dfffbeta4 = dfffbeta4[dfffbeta4['Name'].notnull()]
firstpo = []
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
for i in dfffbeta4['Item'].unique().tolist():    
    tsig = dfffbeta2[dfffbeta2['Item'] == i]
    tsigbeg = tsig[tsig['Customer'] == 'Beginning Inventory']
    tsigbeginv = tsigbeg[~tsigbeg['Location'].isin(InventorySite)]#filter out InventorySite
    totalq = sum(tsigbeginv['quantity'])
    tsigtran = tsig[tsig['Customer'] != 'Beginning Inventory']
    begin = ['Beginning_Inventory', '','', '','','', '','',i,date.today().strftime("%Y-%m-%d"),0.0,0.0,totalq,totalq,'',1,1,'']
    tsige = tsigtran.append(pd.Series(begin, index=['Customer','P.O.number','Customer_PO','Packing_number','Container_number','Order_number','Document_Number','unit_cost','Item','schedule_date','actual inventory level','forecast inventory level','quantity','forecast','Location','p','r','Type']), ignore_index=True)
    #tsige["Transaction_Type"] = ""

    #for ftc in list(tsige.Customer[(tsige['quantity'] <= 0) & (tsige['Customer']!= 'Beginning_Inventory')].index.tolist()):
    #for ftc in list(tsige.Customer[tsige['Customer'].isin(customer)].index.tolist()):
        #tsige.at[ftc, 'Transaction_Type'] = "Sales"
        #tsige.at[ftc, 'p'] = -1
    #for ftv in list(tsige.Customer[(tsige['quantity'] > 0) & (tsige['Customer']!= 'Beginning_Inventory')].index.tolist()):
        #tsige.at[ftv, 'Transaction_Type'] = "Purchase"
        #tsige.at[ftc, 'p'] = 1

    tsigf = tsige.sort_values(by = ['p','schedule_date','r','P.O.number','Type','Document_Number'])
    tsigg = tsigf[['Customer','P.O.number','Customer_PO','Container_number','Document_Number',"SO Lines",'unit_cost','Item','schedule_date','actual inventory level','forecast inventory level','Type','Status','quantity','forecast','Location',"Quantity Committed","Quantity Picked","Quantity Packed","Quantity Fulfilled",'Back Ordered','p']]
    for f0 in list(tsigg.forecast[tsigg['forecast'] == ''].index.tolist()):
        tsigg.at[f0, 'forecast'] = tsigg.at[f0, 'quantity']
    tsigg['balancequantity'] = tsigg['quantity'] 
    tsigg['balanceforecast'] = tsigg['forecast']
    tsigg1 = tsigg[tsigg['Location'] != 'Transit']
    tsigg1 = tsigg1.reset_index(drop = True)
    tsigg1['Location'] = tsigg1['Location'].fillna('')
    for frstimp in list(tsigg1[tsigg1['Location'].str.lower().str.contains('import')].index.tolist()):
        tsigg1.at[frstimp, 'balancequantity'] = 0
        tsigg1.at[frstimp, 'balanceforecast'] = 0

    for psti in list(tsigg1[tsigg1['Type'] == 'Past 90 Days P.O.'].index.tolist()):
        tsigg1.at[psti, 'balancequantity'] = 0
        tsigg1.at[psti, 'balanceforecast'] = 0



    tsigg1['actual inventory level'] = tsigg1['balancequantity'].cumsum()
    tsigg1['forecast inventory level'] = tsigg1['balanceforecast'].cumsum()

    tsigh = tsigg1[['Customer','P.O.number','Customer_PO','Container_number','Document_Number',"SO Lines",'unit_cost','Item','Location','schedule_date','quantity','actual inventory level',"Quantity Committed","Quantity Picked","Quantity Packed","Quantity Fulfilled",'Back Ordered','Type',"Status",'forecast','forecast inventory level']]
    tsigh.columns =['Customer','P.O. Number','NAI P.O.','Container Number','Document Number',"S.O. Lines",'Unit Cost','Item','Location',
                    'Schedule Date',
                    'Quantity','Actual Inventory Level',"Quantity Committed","Quantity Picked","Quantity Packed","Quantity Fulfilled",
                    'Back Ordered','Type',"Status",'Forecast','Forecast Inventory Level']


    #tsigh['Schedule Date'] = pd.to_datetime(tsigh['Schedule Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
    #tsigh['Schedule Date'] = pd.to_datetime(tsigh['Schedule Date'], format='%Y-%m-%d').dt.date
    tsigh = tsigh.reset_index(drop=True)

    try:
        tsighoo = tsigh[tsigh['Quantity']<0]
        onorder = tsighoo['Quantity'].sum() * (-1)
    except:
        onorder = 0

    tsighfp = tsigh
    tsighfp['Schedule Date'] = pd.to_datetime(tsighfp['Schedule Date'], format='%Y-%m-%d')
    tsighfp1 = tsighfp[(tsighfp['Type'].str.contains('P.O'))&(tsighfp['Schedule Date']>date.today().strftime("%Y-%m-%d"))]
    tsighfp1['Schedule Date'] = tsighfp1['Schedule Date'].dt.strftime('%Y-%m-%d')
    try:
        firstpdate = tsighfp1['Schedule Date'].tolist()[0]
    except:
        firstpdate = ''

    firstpo.append((i,firstpdate))

    #################################
    tsighds = tsigh[tsigh['Location']!= 'Import']
    tsighdsa = tsighds[['Schedule Date','Actual Inventory Level']]
    tsighdsf = tsighds[['Schedule Date','Forecast Inventory Level']]
    tsighdsanz = tsighdsa[tsighdsa['Actual Inventory Level']<0].reset_index(drop=True)
    tsighdsfnz = tsighdsf[tsighdsf['Forecast Inventory Level']<0].reset_index(drop=True)


    ########################
    try:
        act_date = tsighdsanz['Schedule Date'][0]
    except:
        act_date = ''

    try:
        fcst_date = tsighdsfnz['Schedule Date'][0]
    except:
        fcst_date = ''
    #############################
    Inv_date_list = [(str(i),act_date,fcst_date,onorder)] 
    Inv_date_pd = pd.DataFrame(Inv_date_list, columns = invcolumns) 
    invdate = invdate.append(Inv_date_pd)    

    tsigh['Schedule Date'] = tsigh['Schedule Date'].dt.strftime('%Y-%m-%d')
    tsigh.to_excel(writer,sheet_name = str(i), startrow=12, startcol=0,index=False,freeze_panes=(13,0))
writer.save()

wme = pd.read_excel('WM Exclusive.xlsx')
wme['Name'] = wme['Name'].astype(str)
wmelist = wme.Name.to_list()

thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

itlst2 = itlst1[['Product Group','Name']]
pglist = itlst2['Product Group'].unique().tolist()

wb2 = load_workbook(file_name) 
try:
    sheet = wb2['Navigation']
except:
    wb2.create_sheet('Navigation',0)
    sheet = wb2['Navigation']
sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=2)
sheet.cell(row=1, column=1).value = 'Navigation Page'
sheet.cell(row=1, column=1).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
sheet.cell(row=1, column=1).font = openpyxl.styles.Font(bold=True, italic=False,color = 'e30000')

sheet.merge_cells(start_row=1, start_column=14, end_row=1, end_column=16)
sheet.cell(row=1, column=14).value = 'Inventory Schedule Report'
sheet.cell(row=1, column=14).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
sheet.cell(row=1, column=14).font = openpyxl.styles.Font(bold=True, italic=False, underline="single")

sheet.merge_cells(start_row=2, start_column=13, end_row=2, end_column=14)
sheet.cell(row=2, column=13).value = 'BlackstoneProduct'
sheet.cell(row=2, column=13).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
sheet.cell(row=2, column=13).font = openpyxl.styles.Font(bold=True, italic=False, underline="none")

sheet.merge_cells(start_row=2, start_column=15, end_row=2, end_column=17)
sheet.cell(row=2, column=15).value = 'North Atlantic Imports LLC'
sheet.cell(row=2, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
sheet.cell(row=2, column=15).font = openpyxl.styles.Font(bold=True, italic=False, underline="none")

sheet.freeze_panes = 'A5'

for pgit in pglist:
    sheet.cell(row=4, column= 2 * pglist.index(pgit) +1).value = pgit
    sheet.cell(row=4, column= 2 * pglist.index(pgit) +1).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=True)
    if '01' in str(pgit):
        sheet.cell(row=4, column= 2 * pglist.index(pgit) +1).font = openpyxl.styles.Font(bold=True, italic=False, underline="none",color = 'c25000')
    else:
        sheet.cell(row=4, column= 2 * pglist.index(pgit) +1).font = openpyxl.styles.Font(bold=True, italic=False, underline="none",color = '006303')
    for itename in itlst2[itlst2['Product Group']== pgit]['Name'].unique().tolist():
        link = file_name +'#' + str(itename) + '!B1'
        sheet.cell(row=5 + itlst2[itlst2['Product Group']== pgit]['Name'].unique().tolist().index(itename) , column= 2 * pglist.index(pgit) +1).value = itename
        sheet.cell(row=5 + itlst2[itlst2['Product Group']== pgit]['Name'].unique().tolist().index(itename) , column= 2 * pglist.index(pgit) +1).hyperlink = (link)
        if str(itename) in wmelist:
            sheet.cell(row=5 + itlst2[itlst2['Product Group']== pgit]['Name'].unique().tolist().index(itename) , column= 2 * pglist.index(pgit) +1).font = openpyxl.styles.Font(bold=True,underline="single",color = 'b00000')
        else:
            sheet.cell(row=5 + itlst2[itlst2['Product Group']== pgit]['Name'].unique().tolist().index(itename) , column= 2 * pglist.index(pgit) +1).font = openpyxl.styles.Font(color = '004880')
################################################################################################# Possibly Here #####################################################################################################################
wb2.save(file_name)

invdate['act_days'] = (pd.to_datetime(invdate['act Inv <0 date'], format='%Y-%m-%d') - pd.to_datetime(dt.datetime.today().strftime('%Y-%m-%d'), format='%Y-%m-%d')).dt.days
invdate['frcst_days'] = (pd.to_datetime(invdate['frcst Inv <0 date'], format='%Y-%m-%d') - pd.to_datetime(dt.datetime.today().strftime('%Y-%m-%d'), format='%Y-%m-%d')).dt.days
invdate = invdate.reset_index(drop = True)

ytd = pd.read_csv("stockledgerYearTD.csv",skiprows = 6)
ytd.columns = ['Item', 'Location', 'Description', 'Class', 'Department',
       'Subsidiary', 'Beginning Inv Qty On-hand', 'Beginning Average Cost',
       'Beginning Inv On-hand Value', 'Receipts', 'Other Inv Inputs',
       'Total Input Quantity', 'Value of Inputs', 'Last Receipt Date',
       'Sales', 'Other Inv Outputs', 'Total Output Quantity',
       'Value of Outputs', 'LAST SALES DATE', 'Ending Inv Qty On-hand',
       'Ending Average Cost', 'Ending Inv On-hand Value']

ytd['Ending Inv Qty On-hand'] = ytd['Ending Inv Qty On-hand'].astype(str)
ytd['Sales'] = ytd['Sales'].astype(str)
for yt in ytd.index.tolist():
    ytd.at[yt, 'Ending Inv Qty On-hand'] = ytd.at[yt, 'Ending Inv Qty On-hand'].replace(',','')
    ytd.at[yt, 'Ending Inv Qty On-hand'] = ytd.at[yt, 'Ending Inv Qty On-hand'].replace('nan','0')
    ytd.at[yt, 'Sales'] = ytd.at[yt, 'Sales'].replace(',','')
    ytd.at[yt, 'Sales'] = ytd.at[yt, 'Sales'].replace('nan','0')    

ytd['Ending Inv Qty On-hand'] = ytd['Ending Inv Qty On-hand'].astype(int)
ytd['Sales'] = ytd['Sales'].astype(int)


mtd = beginninginventory
itemlist = pd.read_csv("itemlist.csv")

ytds = ytd[['Item','Location','Sales']]
ytddoms = ytds[~ytds['Location'].isin(['Import','Transit','Damaged Location'])]
ytddom = ytddoms[ytddoms['Item'].isin(products)]
ytddomaggregate = ytddom.groupby(['Item'],as_index=False).sum()

mtds = mtd[['Item','Location','Sales']]
mtddoms = mtds[~mtds['Location'].isin(['Import','Transit','Damaged Location'])]
mtddom = mtddoms[mtddoms['Item'].isin(products)]
mtddomaggregate = mtddom.groupby(['Item'],as_index=False).sum()

ytdbiao = ytd[['Item','Location','Ending Inv Qty On-hand']]
ytdbiao.columns = ['Item','Location','EndQty']
ytdbiao = ytd[['Item','Location','Ending Inv Qty On-hand']]
ytdbiao.columns = ['Item','Location','EndQty']
ytd1t = ytdbiao[ytdbiao['Location']=="Transit"]
ytd1i = ytdbiao[ytdbiao['Location']=="Import"]
ytd1d = ytdbiao[ytdbiao['Location']=="Damaged Location"]


itemlist['Name'] = itemlist['Name'].astype(str)
itemlist['Display Name'] = itemlist['Display Name'].astype(str)

itemf = pd.read_csv('ItemFulfillmentsforIW.csv')
itemf['IR Date'] = pd.to_datetime(itemf['IR Date'], format='%m/%d/%Y')
itemf['yrmth'] = itemf['IR Date'].dt.to_period('M')
itemf['yrmth'] = itemf['yrmth'].astype(str)
itemfipytd = itemf[itemf['Vendor'].str.lower().str.contains('import')]
itemfipytdag = itemfipytd.groupby(['Item'],as_index=False).sum()

itemfdytd = itemf[~itemf['Vendor'].str.lower().str.contains('import')]
itemfdytdag = itemfdytd.groupby(['Item'],as_index=False).sum()

itemfipmtd = itemfipytd[itemfipytd['yrmth'] == str(datetime.today().strftime("%Y-%m"))].groupby(['Item'],as_index=False).sum()
itemfdmtd = itemfdytd[itemfdytd['yrmth'] == str(datetime.today().strftime("%Y-%m"))].groupby(['Item'],as_index=False).sum()

lyr = pd.read_excel('LastYearSales.xlsx')
lyrip = lyr[lyr['Vendor'].str.lower().str.contains('import')].groupby(['Item'],as_index=False).sum()
lyrdo = lyr[~lyr['Vendor'].str.lower().str.contains('import')].groupby(['Item'],as_index=False).sum()

dfffbeta2po = dfffbeta2[dfffbeta2['Type'].isin(['P.O.','Confirmed P.O.'])]
dfffbeta2po1 = dfffbeta2po[['Item','quantity']]
dfffbeta2po2 = dfffbeta2po1.groupby(['Item'],as_index = False).sum()


firstpo2 = []

#####################################################################################################Tab Color###########################################################################################

book = openpyxl.load_workbook(file_name)
for i in dfffbeta4['Item'].unique().tolist():
    sheet = book[str(i)]
    for itmlst in list(itemlist.Name[itemlist['Name'] == i].index.tolist()):
        itemname = itemlist.at[itmlst, 'Display Name']
        baseprice = itemlist.at[itmlst, 'Base Price']
        itmclass = itemlist.at[itmlst, 'Product Group']
        Pyr = itemlist.at[itmlst, 'Program Year']
        itmstatus = itemlist.at[itmlst, 'Item Status']
    if 'Griddle' in str(itmclass):
        sheet.sheet_properties.tabColor = 'FF9900'
    tsigt = df[(df.Item == i) & (df.Customer == 'Beginning Inventory')]

    if str(i) in wmelist:
        sheet.cell(row=1, column=3).value = 'Walmart Exclusive Item'
        sheet.cell(row=1, column=3).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
        sheet.cell(row=1, column=3).font = openpyxl.styles.Font(bold=True, italic=False,color = 'e61717')
        #sheet.cell(row=1, column=3).style.font.color = 'FF9900'
        #ws = book.worksheets[int(i)]
        #_cell = sheet.cell(row=1, column=3)
        #_cell.style.font.color.index = Color.RED
        


    sheet.cell(row=1, column=1).value = 'Item Number:'
    sheet.cell(row=1, column=1).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=1, column=1).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=1, column=2).value = i
    sheet.cell(row=1, column=2).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=1, column=2).font = openpyxl.styles.Font(bold=True, italic=False)

    sheet.merge_cells(start_row=2, start_column=1, end_row=2, end_column=4)
    sheet.cell(row=2, column=1).value = itemname
    sheet.cell(row=2, column=1).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=1).font = openpyxl.styles.Font(bold=True, italic=False)

    sheet.cell(row=5, column=1).value = "Product Group: "
    sheet.cell(row=5, column=1).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=1).font = openpyxl.styles.Font(bold=False, italic=False)    
    sheet.cell(row=5, column=2).value = itmclass
    sheet.cell(row=5, column=2).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=2).font = openpyxl.styles.Font(bold=True, italic=False) 

    sheet.cell(row=6, column=1).value = "Program Year: "
    sheet.cell(row=6, column=1).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=1).font = openpyxl.styles.Font(bold=False, italic=False)    
    sheet.cell(row=6, column=2).value = Pyr
    sheet.cell(row=6, column=2).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=2).font = openpyxl.styles.Font(bold=True, italic=False)

    sheet.cell(row=7, column=1).value = "Item Status: "
    sheet.cell(row=7, column=1).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=1).font = openpyxl.styles.Font(bold=False, italic=False)    
    sheet.cell(row=7, column=2).value = itmstatus
    sheet.cell(row=7, column=2).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=2).font = openpyxl.styles.Font(bold=True, italic=False)

    Navglink = file_name +'#' + 'Navigation' + '!A1'
    sheet.cell(row=9, column=1).border = thin_border
    sheet.cell(row=9, column=2).border = thin_border
    sheet.cell(row=10, column=1).border = thin_border
    sheet.cell(row=10, column=2).border = thin_border
    sheet.merge_cells(start_row=9, start_column=1, end_row=10, end_column=2)
    sheet.cell(row=9, column=1).value = "Back to Navigation"
    sheet.cell(row=9, column=1).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=1).font = openpyxl.styles.Font(bold=True, italic=False,color = 'e30000')
    sheet.cell(row=9, column=1).hyperlink = (Navglink)


    sheet.merge_cells(start_row=1, start_column=10, end_row=1, end_column=13)
    sheet.cell(row=1, column=10).value = 'Inventory Schedule Report'
    sheet.cell(row=1, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=1, column=10).font = openpyxl.styles.Font(bold=True, italic=False, underline="single")

    sheet.merge_cells(start_row=2, start_column=10, end_row=2, end_column=11)
    sheet.cell(row=2, column=10).value = 'BlackstoneProduct'
    sheet.cell(row=2, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=10).font = openpyxl.styles.Font(bold=True, italic=False, underline="none")

    sheet.merge_cells(start_row=2, start_column=12, end_row=2, end_column=14)
    sheet.cell(row=2, column=12).value = 'North Atlantic Imports LLC'
    sheet.cell(row=2, column=12).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=12).font = openpyxl.styles.Font(bold=True, italic=False, underline="none")

    sheet.cell(row=1, column=18).value = 'Run Date: '
    sheet.cell(row=1, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=1, column=18).font = openpyxl.styles.Font(bold=False, italic=False)
    #sheet.merge_cells(start_row=1, start_column=14, end_row=1, end_column=15)
    sheet.cell(row=1, column=19).value = dt.datetime.today().strftime("%B-%d-%Y")
    sheet.cell(row=1, column=19).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=1, column=19).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=2, column=18).value = 'Run Time: '
    sheet.cell(row=2, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=18).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=2, column=19).value = dt.datetime.today().strftime("%H:%M:%S")
    sheet.cell(row=2, column=19).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=19).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.merge_cells(start_row=2, start_column=20, end_row=2, end_column=22)
    sheet.cell(row=2, column=20).value = str(time.tzname[0])
    sheet.cell(row=2, column=20).alignment = openpyxl.styles.Alignment(horizontal='left', vertical='center', wrap_text=False)
    sheet.cell(row=2, column=20).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=4, column=7).value = 'Current Instock by Location:'
    sheet.cell(row=4, column=7).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=4, column=7).font = openpyxl.styles.Font(bold=True, italic=False)

    sheet.cell(row=5, column=6).value = 'BPUT1:'
    sheet.cell(row=5, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=6).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=5, column=7).value = tsigt.quantity[tsigt['Location']=="BPUT1"].values[0].astype(int)
    except:
        sheet.cell(row=5, column=7).value = 0
    sheet.cell(row=5, column=7).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=7).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=6, column=6).value = "BPUT2:"
    sheet.cell(row=6, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=6).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=6, column=7).value = tsigt.quantity[tsigt['Location']=="BPUT2"].values[0].astype(int)
        sheet.cell(row=6, column=7).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
        sheet.cell(row=6, column=7).font = openpyxl.styles.Font(bold=False, italic=False)
    except:
        sheet.cell(row=6, column=7).value = 0
        sheet.cell(row=6, column=7).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
        sheet.cell(row=6, column=7).font = openpyxl.styles.Font(bold=False, italic=False)    
    
    sheet.cell(row=5, column=10).value = "Memphis (3PL):"
    sheet.cell(row=5, column=10).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=10).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=5, column=11).value = tsigt.quantity[tsigt['Location']=="Memphis (3PL)"].values[0].astype(int)
    except:
        sheet.cell(row=5, column=11).value = 0
    sheet.cell(row=5, column=11).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=11).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=5, column=14).value = "Ogden:"
    sheet.cell(row=5, column=14).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=14).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=5, column=15).value = tsigt.quantity[tsigt['Location']=="Ogden"].values[0].astype(int)
    except:
        sheet.cell(row=5, column=15).value = 0
    sheet.cell(row=5, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=15).font = openpyxl.styles.Font(bold=False, italic=False)       
    
    sheet.cell(row=6, column=14).value = "Drew Logistics LLC:"
    sheet.cell(row=6, column=14).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=14).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=6, column=15).value = tsigt.quantity[tsigt['Location']=="Drew Logistics LLC"].values[0].astype(int)
        sheet.cell(row=6, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
        sheet.cell(row=6, column=15).font = openpyxl.styles.Font(bold=False, italic=False)
    except:
        sheet.cell(row=6, column=15).value = 0
        sheet.cell(row=6, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
        sheet.cell(row=6, column=15).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=6, column=10).value = "CA 3PL:"
    sheet.cell(row=6, column=10).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=10).font = openpyxl.styles.Font(bold=False, italic=False, underline="single")
    try:
        sheet.cell(row=6, column=11).value = tsigt.quantity[tsigt['Location']=="CA 3PL"].values[0].astype(int)
    except:
        sheet.cell(row=6, column=11).value = 0
    sheet.cell(row=6, column=11).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=11).font = openpyxl.styles.Font(bold=False, italic=False)        

    sheet.cell(row=7, column=12).value = "Damaged Location:"
    sheet.cell(row=7, column=12).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=12).font = openpyxl.styles.Font(bold=False, italic=True, underline="double")
    try:
        sheet.cell(row=7, column=13).value = tsigt.quantity[tsigt['Location']=="Damaged Location BP"].values[0].astype(int)

    except:
        sheet.cell(row=7, column=13).value = 0
    sheet.cell(row=7, column=13).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=13).font = openpyxl.styles.Font(bold=False, italic=False)  

    sheet.cell(row=4, column=18).value = 'Total Onhand:'
    sheet.cell(row=4, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=4, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")

    total = 0
    for w in [v for v in Warehouse if v not in InventorySite]:
        try:
            total += tsigt.quantity[tsigt['Location']== w].values[0].astype(int)
        except:
            total += 0

    sheet.cell(row=4, column=19).value = total
    sheet.cell(row=4, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=4, column=19).font = openpyxl.styles.Font(bold=False, italic=False)

    for oodr in list(invdate[invdate['Item'] == str(i)].index.tolist()):
        onorder = invdate.at[oodr, 'on order']

    sheet.cell(row=5, column=18).value = 'Sales on Order:'
    sheet.cell(row=5, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")
    try:
        sheet.cell(row=5, column=19).value = onorder

    except:
        sheet.cell(row=5, column=19).value = 0
    sheet.cell(row=5, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=5, column=19).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=6, column=18).value = 'Purchase on Order:'
    sheet.cell(row=6, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")
    try:
        sheet.cell(row=6, column=19).value = dfffbeta2po2.quantity[ dfffbeta2po2['Item']== str(i)].values[0].astype(int)

    except:
        sheet.cell(row=6, column=19).value = 0
    sheet.cell(row=6, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=6, column=19).font = openpyxl.styles.Font(bold=False, italic=False)


    sheet.cell(row=7, column=18).value = 'Import:'
    sheet.cell(row=7, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")
    
    try:
        sheet.cell(row=7, column=19).value = ytd1i.EndQty[ytd1i['Item']== str(i)].values[0].astype(int)

    except:
        sheet.cell(row=7, column=19).value = 0
    sheet.cell(row=7, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=7, column=19).font = openpyxl.styles.Font(bold=False, italic=False)

    sheet.cell(row=8, column=18).value = 'Transit:'
    sheet.cell(row=8, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=8, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")
    try:
        sheet.cell(row=8, column=19).value = shipment4.quantity[shipment4['Item']== str(i)].values[0].astype(int)
    except:
        sheet.cell(row=8, column=19).value = 0
    sheet.cell(row=8, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=8, column=19).font = openpyxl.styles.Font(bold=False, italic=False)
    
    available = total - onorder
    sheet.cell(row=9, column=18).value = "Available:"
    sheet.cell(row=9, column=18).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=18).font = openpyxl.styles.Font(bold=False, italic=False, underline="double")    
    sheet.cell(row=9, column=19).value = max([0,available])
    sheet.cell(row=9, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=19).font = openpyxl.styles.Font(bold=False, italic=False)
    
    
    sheet.cell(row=8, column=15).value = 'Import'
    sheet.cell(row=8, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=8, column=15).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=8, column=15).border = thin_border

    sheet.cell(row=8, column=16).value = 'Domestic'
    sheet.cell(row=8, column=16).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=8, column=16).font = openpyxl.styles.Font(bold=False, italic=False) 
    sheet.cell(row=8, column=16).border = thin_border

    sheet.cell(row=8, column=17).value = 'Total'
    sheet.cell(row=8, column=17).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=8, column=17).font = openpyxl.styles.Font(bold=False, italic=False) 
    sheet.cell(row=8, column=17).border = thin_border

    for lyipt in list(lyrip.Item[lyrip['Item'] == str(i)].index.tolist()):
        lyimport = lyrip.at[lyipt, 'IR Quantity']

    for lyd in list(lyrdo.Item[lyrdo['Item'] == str(i)].index.tolist()):
        lydo = lyrdo.at[lyd, 'IR Quantity']      


    sheet.cell(row=8, column=14).border = thin_border    
    sheet.cell(row=9, column=14).value = 'LY_Shipped'
    sheet.cell(row=9, column=14).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=14).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=9, column=14).border = thin_border
    try:
        sheet.cell(row=9, column=15).value = lyimport
    except:
        sheet.cell(row=9, column=15).value = 0
    sheet.cell(row=9, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=15).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=9, column=15).border = thin_border
    try:
        sheet.cell(row=9, column=16).value = lydo
    except:
        sheet.cell(row=9, column=16).value = 0
    sheet.cell(row=9, column=16).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=16).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=9, column=16).border = thin_border
    try:
        sheet.cell(row=9, column=17).value = lydo + lyimport
    except:
        sheet.cell(row=9, column=17).value = 0
    sheet.cell(row=9, column=17).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=17).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=9, column=17).border = thin_border

    lyimport = 0
    lydo = 0


    for mtdqn in list(itemfipmtd.Item[itemfipmtd['Item'] == str(i)].index.tolist()):
        mtdqnimport = itemfipmtd.at[mtdqn, 'IR Quantity']

    for ytdqn in list(itemfipytdag.Item[itemfipytdag['Item'] == str(i)].index.tolist()):
        ytdimport = itemfipytdag.at[ytdqn, 'IR Quantity']    

    for mtdqnd in list(itemfdmtd.Item[itemfdmtd['Item'] == str(i)].index.tolist()):
        mtdqndo = itemfdmtd.at[mtdqnd, 'IR Quantity']

    for ytdqnd in list(itemfdytdag.Item[itemfdytdag['Item'] == str(i)].index.tolist()):
        ytdimdo = itemfdytdag.at[ytdqnd, 'IR Quantity'] 



    sheet.cell(row=10, column=14).value = 'YTD_Shipped'
    sheet.cell(row=10, column=14).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=14).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=14).border = thin_border
    try:
        sheet.cell(row=10, column=15).value = ytdimport
    except:
        sheet.cell(row=10, column=15).value = 0
    sheet.cell(row=10, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=15).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=15).border = thin_border
    try:
        sheet.cell(row=10, column=16).value = ytdimdo
    except:
        sheet.cell(row=10, column=16).value = 0
    sheet.cell(row=10, column=16).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=16).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=16).border = thin_border
    try:
        sheet.cell(row=10, column=17).value = ytdimdo + ytdimport
    except:
        sheet.cell(row=10, column=17).value = 0
    sheet.cell(row=10, column=17).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=17).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=17).border = thin_border
    ytdimport = 0
    ytdimdo = 0    

    sheet.cell(row=11, column=14).value = 'MTD_Shipped'
    sheet.cell(row=11, column=14).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=14).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=11, column=14).border = thin_border
    try:
        sheet.cell(row=11, column=15).value = mtdqnimport

    except:
        sheet.cell(row=11, column=15).value = 0
    sheet.cell(row=11, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=15).font = openpyxl.styles.Font(bold=False, italic=False) 
    sheet.cell(row=11, column=15).border = thin_border
    try:
        sheet.cell(row=11, column=16).value = mtdqndo

    except:
        sheet.cell(row=11, column=16).value = 0
    sheet.cell(row=11, column=16).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=16).font = openpyxl.styles.Font(bold=False, italic=False) 
    sheet.cell(row=11, column=16).border = thin_border
    try:
        sheet.cell(row=11, column=17).value = mtdqndo + mtdqnimport

    except:
        sheet.cell(row=11, column=17).value = 0
    sheet.cell(row=11, column=17).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=17).font = openpyxl.styles.Font(bold=False, italic=False)     
    sheet.cell(row=11, column=17).border = thin_border
    mtdqndo = 0
    mtdqnimport = 0     
#############################################    

    #for basep in list(itemlist.Name[itemlist['Name'] == str(i)].index.tolist()):
        #baseprice = itemlist.at[basep, 'Base Price']
    for pp in list(itemlist.Name[itemlist['Name'] == str(i)].index.tolist()):
        purchaseprice = itemlist.at[pp, 'Purchase Price']  
    for lastpp in list(itemlist.Name[itemlist['Name'] == str(i)].index.tolist()):
        lastpurchaseprice = itemlist.at[lastpp, 'Last Purchase Price']    

    #sheet.cell(row=9, column=5).value = 'Base_Price'
    #sheet.cell(row=9, column=5).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    #sheet.cell(row=9, column=5).font = openpyxl.styles.Font(bold=False, italic=False)
    #try:
        #sheet.cell(row=9, column=6).value = baseprice
        #sheet.cell(row=9, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        #sheet.cell(row=9, column=6).font = openpyxl.styles.Font(bold=False, italic=False)
    #except:
        #sheet.cell(row=9, column=6).value = 'Nan'
        #sheet.cell(row=9, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        #sheet.cell(row=9, column=6).font = openpyxl.styles.Font(bold=False, italic=False) 

    sheet.cell(row=10, column=5).value = 'Purchase_Price:'
    sheet.cell(row=10, column=5).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=5).font = openpyxl.styles.Font(bold=False, italic=False)
    try:
        sheet.cell(row=10, column=6).value = purchaseprice
        sheet.cell(row=10, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        sheet.cell(row=10, column=6).font = openpyxl.styles.Font(bold=False, italic=False)
    except:
        sheet.cell(row=10, column=6).value = 'Nan'
        sheet.cell(row=10, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        sheet.cell(row=10, column=6).font = openpyxl.styles.Font(bold=False, italic=False)    

    sheet.cell(row=11, column=5).value = 'Last_Purchase_Price:'
    sheet.cell(row=11, column=5).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=5).font = openpyxl.styles.Font(bold=False, italic=False)
    try:
        sheet.cell(row=11, column=6).value = lastpurchaseprice
        sheet.cell(row=11, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        sheet.cell(row=11, column=6).font = openpyxl.styles.Font(bold=False, italic=False)
    except:
        sheet.cell(row=11, column=6).value = "Nan"
        sheet.cell(row=11, column=6).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
        sheet.cell(row=11, column=6).font = openpyxl.styles.Font(bold=False, italic=False) 

    for act_date1 in list(invdate[invdate['Item'] == str(i)].index.tolist()):
        act_date = invdate.at[act_date1, 'act Inv <0 date']
    for fsct_date1 in list(invdate[invdate['Item'] == str(i)].index.tolist()):
        fsct_date = invdate.at[act_date1, 'frcst Inv <0 date']       
    for act_day1 in list(invdate[invdate['Item'] == str(i)].index.tolist()):
        act_day = invdate.at[act_day1, 'act_days']        
    for frst_day1 in list(invdate[invdate['Item'] == str(i)].index.tolist()):
        frst_day = invdate.at[frst_day1, 'frcst_days']

    sheet.cell(row=9, column=8).border = thin_border
    sheet.cell(row=10, column=8).value = 'Actual'
    sheet.cell(row=10, column=8).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=8).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=8).border = thin_border
    sheet.cell(row=11, column=8).value = 'Forecast'
    sheet.cell(row=11, column=8).alignment = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=8).font = openpyxl.styles.Font(bold=False, italic=False)  
    sheet.cell(row=11, column=8).border = thin_border

    sheet.cell(row=9, column=9).value = 'Stock below 0 Date'
    sheet.cell(row=9, column=9).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=9).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=9, column=9).border = thin_border
    sheet.cell(row=9, column=10).value = 'Days'
    sheet.cell(row=9, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=9, column=10).font = openpyxl.styles.Font(bold=False, italic=False)     
    sheet.cell(row=9, column=10).border = thin_border
    try:
        sheet.cell(row=10, column=9).value = act_date
    except:
        sheet.cell(row=10, column=9).value = ""
    sheet.cell(row=10, column=9).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=9).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=9).border = thin_border

    try:
        sheet.cell(row=11, column=9).value = fsct_date

    except:
        sheet.cell(row=11, column=9).value = ""
    sheet.cell(row=11, column=9).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=9).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=11, column=9).border = thin_border

    try:
        sheet.cell(row=10, column=10).value = act_day

    except:
        sheet.cell(row=10, column=10).value = ""
    sheet.cell(row=10, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=10, column=10).font = openpyxl.styles.Font(bold=False, italic=False)
    sheet.cell(row=10, column=10).border = thin_border

    try:
        sheet.cell(row=11, column=10).value = frst_day

    except:
        sheet.cell(row=11, column=10).value = ""
    sheet.cell(row=11, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=False)
    sheet.cell(row=11, column=10).font = openpyxl.styles.Font(bold=False, italic=False)     
    sheet.cell(row=11, column=10).border = thin_border

    firstpo2.append((i,itemname,itmstatus,sheet.cell(row=4, column=19).value,sheet.cell(row=5, column=19).value,sheet.cell(row=6, column=19).value,sheet.cell(row=7, column=19).value,sheet.cell(row=8, column=19).value,sheet.cell(row=9, column=19).value))
    
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 15
    sheet.column_dimensions['C'].width = 7
    sheet.column_dimensions['D'].width = 13
    sheet.column_dimensions['E'].width = 11
    sheet.column_dimensions['F'].width = 6
    sheet.column_dimensions['G'].width = 7
    sheet.column_dimensions['H'].width = 6
    sheet.column_dimensions['I'].width = 18
    sheet.column_dimensions['J'].width = 10
    sheet.column_dimensions['K'].width = 9
    sheet.column_dimensions['L'].width = 10
    sheet.column_dimensions['M'].width = 11
    sheet.column_dimensions['N'].width = 9
    sheet.column_dimensions['O'].width = 9
    sheet.column_dimensions['P'].width = 9
    sheet.column_dimensions['Q'].width = 9
    sheet.column_dimensions['R'].width = 25
    sheet.column_dimensions['S'].width = 18
    sheet.column_dimensions['T'].width = 9
    sheet.column_dimensions['U'].width = 10

    sheet.cell(row=13, column=1).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=2).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=3).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=4).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=5).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=6).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=7).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=8).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=9).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=10).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=11).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=12).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=13).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=14).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=15).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=16).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=17).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=18).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=19).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=20).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
    sheet.cell(row=13, column=21).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)

book.save(file_name)

firstpodf = pd.DataFrame(firstpo,columns =['Item', 'Next Purchase Date'])
firstpo2df = pd.DataFrame(firstpo2,columns =['Item', 'Display Name','Status','Onhand','Sales on Order','Purchase on Order','Import','Transit','Available'])
cis = pd.merge(firstpo2df,firstpodf,on = 'Item')
invdate = invdate.sort_values(by = ['act_days','frcst_days','Item'],na_position='last')
cis = cis.sort_values(by = 'Item')
cis.to_excel('Current_Inventory_Status.xlsx',index = False)
invdate.to_excel('Inventory_Date.xlsx',index = False)

<ipython-input-2-962507a59dc7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beginninginventory2['Customer'] = 'Beginning Inventory'
<ipython-input-2-962507a59dc7>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bi['Customer'] = 'Beginning Inventory'
<ipython-input-2-962507a59dc7>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-